### Reply_Crawler

댓글 가져오는 정보정리 
네이버, 틱톡 = API가져와서 하면 가능함 


In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time 

import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [14]:
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
# options.add_argument('--headless') 
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ChromeDriver의 경로를 명시적으로 지정합니다.
driver = webdriver.Chrome(options=options)

# URL 입력하는곳
########################################
# driver.get('https://youtu.be/qWbHSOplcvY')
########################################
# URL 붙여넣기로 교체
driver.get(input())

### 겜톡톡 ###
# https://www.youtube.com/shorts/V33wIascWBE
#######################################

driver.implicitly_wait(3)
time.sleep(1.5)

##이 단계에서 버튼 눌러서 댓글창 열리게 만들기 


driver.execute_script('window.scrollTo(0,800)')  # 800픽셀만큼 페이지 내리기
time.sleep(3)

# 필요한 작업 수행 후 드라이버 종료
# driver.quit() 


 https://www.youtube.com/shorts/V33wIascWBE


In [15]:
last_height = driver.execute_script("return document.documentElement.scrollHeight") 
# 최초 로드시 화면이 가진 스크롤 높이 최대 픽셀값을 추출

while True: # for 문과 달리 무한 루프인데 조건을 충족하면 코드를 벗어남
  driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
  # 지금 보고 있는 화면이 가진 스크롤 높이 최대 픽셀값만큼 실행
  # 즉, 위에서 받은 최대 스크롤 값을 실행하는 것
  # ; = window 함수의 특징으로서 code가 끝났으니 다음으로 넘어가라는 표시
  time.sleep(1.5)

  new_height = driver.execute_script("return document.documentElement.scrollHeight") 
  # 기존에 받아온 최대 스크롤값을 실행한 후 추가로 생긴 화면이 가진 스크롤 높이 최대 픽셀값 추출

  if new_height == last_height: # 추가 스크롤 값 없으면 멈춰
    break

  last_height = new_height # 스크롤 더 있으면 다시 담아줘(last height 리셋)
  time.sleep(1.5)

  try: 
    driver.find_element_by_css_selector('#dismiss-button > a').click() 
    #유튜브 프리미엄 프로모션 팝업창 닫기

    time.sleep(1.5)
  except:
    pass

In [16]:
html_source = driver.page_source 
#print(html_source)
soup = BeautifulSoup(html_source, 'html.parser') #파싱 가능한 형태로 만듦

id_list = soup.select('div#header-author > h3 > #author-text > span') 
# 아이디 리스트 변수(html 태그 리스트를 담은 것)
comment_list = soup.select('yt-formatted-string#content-text') 
# 댓글 리스트 변수(html 태그 리스트를 담은 것)

id_final = [] # 리스트 형태라고 선언
comment_final = [] # 리스트 형태라고 선언

for i in range(len(comment_list)):
  temp_id = id_list[i].text
  temp_id = temp_id.replace('\n','').replace('\t','').replace(' ','').strip()
  id_final.append(temp_id) # 댓글 작성자 담기

  temp_comment = comment_list[i].text
  temp_comment = temp_comment.replace('\n','').replace('\t','').strip()
  comment_final.append(temp_comment)  # 댓글 내용 담기

In [17]:
# 데이터 프레임 만들기 (리스트 -> 딕셔너리 활용 -> 데이터프레임)
youtube_dict = {"아이디": id_final, "댓글 내용": comment_final}
youtube_df = pd.DataFrame(youtube_dict)

In [19]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   아이디     0 non-null      float64
 1   댓글 내용   0 non-null      float64
dtypes: float64(2)
memory usage: 124.0 bytes
